In [6]:
import itk
import nibabel as nib
import numpy as np
import nrrd

ImagePath1 = "Z:/D-Images/SPIROMICS-SubStudy/2-Results-CheckedDoneTemp/1-Done/Case-42-Spiromics-67503688/Markups/88-Adam/S-Final-2-label.nrrd"
ImagePath2 = "Z:/E-Kelly-VidaSegmentals&Sublobes/3- 20Spiromics-10ForKelly-ForERS23/Case42-IA210257_H-17900_20190805_125316_20230524_121115_export/ZUNU_vida-airtree-recentered.nii.gz"
#Try taking image 2 into upsampled geometry of image 1. Will have to resample
#Image 1 is an nrrd and Image 2 is a nii.gz

image1, header1 = nrrd.read(ImagePath1)

image2 = nib.load(ImagePath2)
data = image2.get_fdata()
header2 = image2.header


In [5]:
header1

OrderedDict([('type', 'short'),
             ('dimension', 3),
             ('space', 'left-posterior-superior'),
             ('sizes', array([755, 608, 600])),
             ('space directions',
              array([[0.38, 0.  , 0.  ],
                     [0.  , 0.38, 0.  ],
                     [0.  , 0.  , 0.38]])),
             ('kinds', ['domain', 'domain', 'domain']),
             ('endian', 'little'),
             ('encoding', 'gzip'),
             ('space origin', array([-137., -298., -311.]))])